<a href="https://colab.research.google.com/github/Hllsweb/B.L.A-AI/blob/main/WebScrappingML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Instalação das bibliotecas principais para web scraping avançado
!pip install requests beautifulsoup4 lxml pandas tqdm nest_asyncio

# Instalação do Scrapy (opcional, pode ser usado para crawling profundo)
!pip install scrapy

# (Opcional) Instalação do Selenium para páginas dinâmicas
# !pip install selenium webdriver-manager

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 46.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.8/259.8 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.4/107.4 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.6/74.6 kB 4.3 MB/s eta 0:00:00


In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm
import lxml
import nest_asyncio
import scrapy

print("Todas as bibliotecas foram importadas com sucesso!")


Todas as bibliotecas foram importadas com sucesso!


In [3]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse

# Seeds definidas
seeds = [
    "https://www.mercadolivre.com.br/ajuda/termos-e-condicoes-de-uso_991",
    "https://www.mercadolivre.com.br/ajuda/politica-de-privacidade_881",
    "https://www.mercadolivre.com.br/ajuda/politicas-de-anuncios_1122",
    "https://www.mercadolivre.com.br/ajuda/o-que-e-o-sistema-de-reputacao_220",
    "https://www.mercadolivre.com.br/ajuda",
    "https://www.mercadolivre.com.br/ajuda/vender-no-mercado-livre_604",
    "https://www.mercadolivre.com.br/ajuda/politica-de-devolucao_630"
]

def is_internal(url):
    # Só considera links do domínio mercadolivre.com.br/ajuda
    parsed = urlparse(url)
    return parsed.netloc == "www.mercadolivre.com.br" and parsed.path.startswith("/ajuda")

for seed in seeds:
    print(f"\n\n>>> Página: {seed}")
    try:
        resp = requests.get(seed, timeout=10)
        soup = BeautifulSoup(resp.text, "lxml")

        # Extrai título
        title = soup.find("title").text.strip() if soup.find("title") else "Sem título"
        print(f"Título: {title}\n")

        # Extrai texto principal (melhor heurística inicial: pega todo o texto da <main> ou <body>)
        main = soup.find("main") or soup.find("body")
        text = main.get_text(separator="\n", strip=True) if main else ""
        print(f"Texto (primeiros 400 caracteres):\n{text[:400]}...\n")

        # Extrai links internos únicos
        links = set()
        for a in soup.find_all("a", href=True):
            url = urljoin(seed, a['href'])
            if is_internal(url):
                links.add(url)
        print(f"Links internos encontrados ({len(links)}):")
        for l in list(links)[:10]:  # Mostra só os 10 primeiros
            print(" -", l)
        if len(links) > 10:
            print(" ...")
    except Exception as e:
        print("Erro ao acessar:", e)



>>> Página: https://www.mercadolivre.com.br/ajuda/termos-e-condicoes-de-uso_991
Título: Termos e condições gerais de uso do site

Texto (primeiros 400 caracteres):
Ajuda
Termos e condições gerais de uso do site
Buscar
Termos e condições gerais de uso do site
English Version
Última modificação: 24 de novembro, 2023
Resumo dos Termos e condições
O Mercado Livre é uma empresa de tecnologia e oferece serviços relacionados, principalmente, com  comércio eletrônico e pagamentos digitais.
O Marketplace é uma plataforma de comércio eletrônico onde as Pessoas Usuária...

Links internos encontrados (17):
 - https://www.mercadolivre.com.br/ajuda/Termos-e-condicoes-gerais-de-uso_1409
 - https://www.mercadolivre.com.br/ajuda/Termos-e-condicoes-gerais-de-uso_1500
 - https://www.mercadolivre.com.br/ajuda/30487
 - https://www.mercadolivre.com.br/ajuda/1086
 - https://www.mercadolivre.com.br/ajuda/25781
 - https://www.mercadolivre.com.br/ajuda/299
 - https://www.mercadolivre.com.br/ajuda/termos-e-con

In [4]:
import requests
from bs4 import BeautifulSoup

problematic_urls = [
    "https://www.mercadolivre.com.br/ajuda/politica-de-privacidade_881",
    "https://www.mercadolivre.com.br/ajuda/politicas-de-anuncios_1122",
    "https://www.mercadolivre.com.br/ajuda/politica-de-devolucao_630"
]

headers = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/122.0.0.0 Safari/537.36"
    ),
    "Accept-Language": "pt-BR,pt;q=0.9,en;q=0.8",
    "Accept-Encoding": "gzip, deflate, br",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
    "Connection": "keep-alive"
}

for url in problematic_urls:
    print(f"\n>>> Testando: {url}")
    try:
        resp = requests.get(url, headers=headers, timeout=10)
        soup = BeautifulSoup(resp.text, "lxml")
        title = soup.find("title").text.strip() if soup.find("title") else "Sem título"
        print(f"Título: {title}\n")
        main = soup.find("main") or soup.find("body")
        text = main.get_text(separator="\n", strip=True) if main else ""
        print(f"Texto (primeiros 400 caracteres):\n{text[:400]}...\n")
    except Exception as e:
        print("Erro:", e)


>>> Testando: https://www.mercadolivre.com.br/ajuda/politica-de-privacidade_881
Título: Error

Texto (primeiros 400 caracteres):
...


>>> Testando: https://www.mercadolivre.com.br/ajuda/politicas-de-anuncios_1122
Título: Error

Texto (primeiros 400 caracteres):
...


>>> Testando: https://www.mercadolivre.com.br/ajuda/politica-de-devolucao_630
Título: Error

Texto (primeiros 400 caracteres):
...



In [5]:
# Instalação de dependências do Selenium e ChromeDriver no Colab
!apt-get update
!apt install chromium-chromedriver
!pip install selenium webdriver-manager

import sys
sys.path.insert(0, '/usr/lib/chromium-browser/chromedriver')

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,543 kB]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [3,154 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/restricted am

In [6]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time

urls = [
    "https://www.mercadolivre.com.br/ajuda/politica-de-privacidade_881",
    "https://www.mercadolivre.com.br/ajuda/politicas-de-anuncios_1122",
    "https://www.mercadolivre.com.br/ajuda/politica-de-devolucao_630"
]

options = Options()
options.add_argument("--headless")
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")
options.add_argument("--window-size=1920,1080")
options.add_argument('--disable-dev-shm-usage')
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36")

driver = webdriver.Chrome(options=options)

for url in urls:
    print(f"\n>>> Testando (Selenium): {url}")
    try:
        driver.get(url)
        time.sleep(3)  # Aguarda carregar JavaScript
        title = driver.title
        print(f"Título: {title}")
        # Tenta pegar o texto do <main> ou <body>
        try:
            main = driver.find_element(By.TAG_NAME, "main")
        except:
            main = driver.find_element(By.TAG_NAME, "body")
        text = main.text
        print(f"Texto (primeiros 400 caracteres):\n{text[:400]}...\n")
    except Exception as e:
        print("Erro:", e)

driver.quit()


>>> Testando (Selenium): https://www.mercadolivre.com.br/ajuda/politica-de-privacidade_881
Título: Error
Texto (primeiros 400 caracteres):
Parece que esta página não existe
Se o problema persistir, você pode reportá-lo na seção Ajuda, usando o código a seguir:
Código: 
404 
Copiar
Voltar...


>>> Testando (Selenium): https://www.mercadolivre.com.br/ajuda/politicas-de-anuncios_1122
Título: Error
Texto (primeiros 400 caracteres):
Parece que esta página não existe
Se o problema persistir, você pode reportá-lo na seção Ajuda, usando o código a seguir:
Código: 
404 
Copiar
Voltar...


>>> Testando (Selenium): https://www.mercadolivre.com.br/ajuda/politica-de-devolucao_630
Título: Error
Texto (primeiros 400 caracteres):
Parece que esta página não existe
Se o problema persistir, você pode reportá-lo na seção Ajuda, usando o código a seguir:
Código: 
404 
Copiar
Voltar...



In [7]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse
import pandas as pd
from tqdm import tqdm
import time

# Configurações iniciais
START_URL = "https://www.mercadolivre.com.br/ajuda"
DOMAIN = "www.mercadolivre.com.br"
PATH_PREFIX = "/ajuda"
MAX_DEPTH = 3  # Para evitar loops infinitos (ajuste se necessário)
SLEEP_TIME = 0.7  # Para não sobrecarregar o site

headers = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/122.0.0.0 Safari/537.36"
    ),
    "Accept-Language": "pt-BR,pt;q=0.9,en;q=0.8"
}

def is_internal_url(url):
    parsed = urlparse(url)
    return parsed.netloc == DOMAIN and parsed.path.startswith(PATH_PREFIX)

def is_error_page(soup):
    # Detecta páginas de erro (404, etc)
    if not soup.title:
        return True
    title = soup.title.text.strip().lower()
    if 'error' in title or 'não existe' in title or '404' in title or 'não encontrada' in title:
        return True
    # Pode expandir checagens aqui
    return False

def extract_links(soup, base_url):
    links = set()
    for a in soup.find_all("a", href=True):
        url = urljoin(base_url, a['href'])
        if is_internal_url(url):
            links.add(url.split("#")[0])  # Remove ancoras
    return links

def extract_text(soup):
    main = soup.find("main") or soup.find("body")
    if not main:
        return ""
    text = main.get_text(separator="\n", strip=True)
    return text

# Estruturas de controle
visited = set()
to_visit = [(START_URL, 0)]
data = []

with tqdm(total=1, desc="Crawling páginas") as pbar:
    while to_visit:
        url, depth = to_visit.pop(0)
        if url in visited or depth > MAX_DEPTH:
            continue
        try:
            resp = requests.get(url, headers=headers, timeout=10)
            soup = BeautifulSoup(resp.text, "lxml")
            if is_error_page(soup):
                visited.add(url)
                continue
            title = soup.title.text.strip() if soup.title else "Sem título"
            text = extract_text(soup)
            links = extract_links(soup, url)
            data.append({
                'url': url,
                'titulo': title,
                'texto': text,
                'links_encontrados': list(links),
                'profundidade': depth,
            })
            # Adiciona novos links na fila de visitação
            for link in links:
                if link not in visited:
                    to_visit.append((link, depth+1))
            visited.add(url)
            pbar.total = pbar.total + len(links)
            pbar.update(1)
            time.sleep(SLEEP_TIME)
        except Exception as e:
            visited.add(url)
            continue

# Exporta para DataFrame
df = pd.DataFrame(data)
print(f"\nTotal de páginas coletadas: {len(df)}")
df.head()

# (Opcional) Salva para CSV
df.to_csv("mercadolivre_politicas_crawl.csv", index=False)
print("Arquivo salvo: mercadolivre_politicas_crawl.csv")

Crawling páginas:  14%|█▍        | 233/1656 [03:53<23:46,  1.00s/it]


Total de páginas coletadas: 233
Arquivo salvo: mercadolivre_politicas_crawl.csv


In [9]:
# Leitura do csv no Colab
df = pd.read_csv("mercadolivre_politicas_crawl.csv")

In [10]:
!pip install pandas requests beautifulsoup4 lxml tqdm

In [11]:
!python3 mercadolivre.py

100% 233/233 [03:12<00:00,  1.21it/s]
Extração web finalizada. Arquivo salvo: mercadolivre_politicas_full_extract.csv


In [12]:
!python3 processa_base_ia.py

🔹 Lendo arquivo: mercadolivre_politicas_full_extract.csv
🔹 Fragmentando textos em blocos estruturados...
100% 233/233 [00:00<00:00, 3578.40it/s]
✅ Base de conhecimento estruturada salva em: mercadolivre_base_conhecimento_ia.csv
Total de blocos gerados: 964
                                             url  ...                                        texto_bloco
53     https://www.mercadolivre.com.br/ajuda/299  ...  e)  Aderir a produtos e serviços diretamente n...
857  https://www.mercadolivre.com.br/ajuda/29902  ...  FATTOR RECUPERACAO DE CREDITOS E GESTAO DE RIS...
394  https://www.mercadolivre.com.br/ajuda/28247  ...  x) Animais domésticos ou não, roedores ou inse...

[3 rows x 5 columns]
